Environment

In [ ]:
import os
import sys
sys.path.append("./../")

In [ ]:
from src import env
env.set(
  backend="torch",
  device="cpu",
  device_idx=1,
  nb_threads=16,
  epsilon=1e-10,
  floatx="float64"
)

Import libraries

In [ ]:
import torch
import numpy as np

from src import const
from src.utils import *
from src.systems import TAFASystem
from matplotlib import pyplot as plt

In [ ]:
from scipy import constants
from scipy.constants import physical_constants as pc

UNA = pc["Avogadro constant"][0]
UKB = pc["Boltzmann constant"][0]
eV_to_J = constants.eV

Define inputs

In [ ]:
# System
T = 1e4
ic = {
  "T": 4000,
  "p": 1000,
  "Xa": 0.05
}
atom = "O"
molecule = "O2"
max_mom = 10
# Paths
paths = {
  "dtb": "./../database/VS_O3_O4/",
  "bpod": f"./../data/bpod_lin_tafa.T{int(T)}K.m{max_mom}"
}
# Plotting
saving = True
showing = False

In [ ]:
if saving:
  os.makedirs(paths["bpod"]+"/figs", exist_ok=True)

Initialize isothermal master equation model

In [ ]:
model = TAFASystem(
  rates=paths["dtb"]+"kinetics.hdf5",
  species={k: paths["dtb"]+f"species/{k}.json" for k in ("atom", "molecule")},
  use_einsum=False
)
model.update_fom_ops(T)
sp_mol = model.species["molecule"]

In [ ]:
gi = sp_mol.lev["g"]
ei = sp_mol.lev['e']/const.eV_to_J

FOM solution

In [ ]:
t = np.geomspace(1e-12, 1e-3, num=199)
t = np.insert(t, 0, 0.0)

In [ ]:
n_0 = ic["p"] / (const.UKB * ic["T"])
na_0 = np.array([n_0 * ic["Xa"]]).reshape(-1)
qm_0 = sp_mol.q_int(ic["T"])
nm_0 = n_0 * (1.0-ic["Xa"]) * qm_0 / np.sum(qm_0)

In [ ]:
y0 = np.concatenate([na_0, nm_0])
yf = model.solve(t, y0, ops=model.fom_ops, rtol=1e-7, atol=0.0)
na, nm = yf[:1], yf[1:]

Balanced POD

In [ ]:
s, phi, psi = [
  torch.load(paths["bpod"]+f"/{k}.pt").numpy() for k in ("s", "phi", "psi")
]

In [ ]:
cs = 1.0 / np.sum(s**2)
cs *= np.cumsum(s**2)

In [ ]:
# Number of principal components
eps = 1e-5
romdim = np.where(cs > 1-eps)[0][0]+1
romdim

In [ ]:
dimmax = 20
plot_cumenergy(
  cs[:dimmax],
  filename=paths["bpod"] + "/figs/cum_en.png",
  save=saving,
  show=showing
)

In [ ]:
for i in range(dimmax):
  nb = str(i+1)
  plot_2D(
    x=sp_mol.lev['e']/const.eV_to_J,
    y_true=phi[:,i],
    labels=[r"$\epsilon_i$ [eV]", r"$\psi_{{%s}}$" % nb],
    scales=["linear", "linear"],
    filename=paths["bpod"] + f"/figs/phi_{nb.zfill(2)}.png",
    save=saving,
    show=showing
  )

ROM Model - Testing

In [ ]:
rom_dim = 10

In [ ]:
model.set_basis(phi=phi[:,:rom_dim], psi=psi[:,:rom_dim])
model.update_rom_ops()

In [ ]:
y0 = np.concatenate([na_0, model.psi.T @ nm_0])
yr = model.solve(t, y0, ops=model.rom_ops, rtol=1e-7, atol=0.0)
na_pred = yr[:1]
nm_pred = model.phi @ yr[1:]

FOM vs. ROM

In [ ]:
100 * np.mean(np.abs(na_pred - na) / np.abs(na))

In [ ]:
plt.loglog(t, na.squeeze())
plt.loglog(t, na_pred.squeeze(), ls='--')

In [ ]:
one = np.ones_like(ei)

In [ ]:
m = nm.T @ one
m_pred = nm_pred.T @ one
100 * np.mean(np.abs(m_pred - m) / np.abs(m))

In [ ]:
plt.loglog(t, m.squeeze())
plt.loglog(t, m_pred.squeeze(), ls='--')

In [ ]:
e = nm.T @ ei / m
e_pred = nm_pred.T @ ei / m_pred
100 * np.mean(np.abs(e_pred - e) / np.abs(e))

In [ ]:
45**3/10**3

In [ ]:
plt.loglog(t, e.squeeze())
plt.loglog(t, e_pred.squeeze(), ls='--')

In [ ]:
# Plot distributions
for i in range(0,200,30):
  print("t = ",t[i])
  plot_2D(
    x=ei,
    y_true=nm[:,i] / sp_mol.lev["g"],
    y_pred=nm_pred[:,i] / sp_mol.lev["g"],
    scales=["linear", "log"],
    filename=paths["bpod"] + f"/figs/sol_2d_i{str(i).zfill(4)}_rom.png",
    save=saving,
    show=showing
  )

In [ ]:
i = np.argwhere(t >= 1e-9)[0][0]
j = np.argwhere(t >= 1e-4)[0][0]
y = {
  "FOM": nm.T[i:j] / gi,
  "ROM": nm_pred.T[i:j] / gi
}

In [ ]:
# animate(
#   t=t[i:j],
#   x=ei,
#   y=y,
#   frames=50,
#   fps=10,
#   filename=paths["bpod"] + f"/figs/lev_dist_T0{int(ic['T'])}K_r{rom_dim}.gif",
#   save=saving,
#   show=showing
# )